In [ ]:
import numpy as np
from pandas import pandas as pd
import tensorflow as tf
from enum import Enum

In [ ]:
class BatchNormWrapper():
    def __init__(self, useBatchNorm = False, offset = 0, scale = 1, var_eps = 1e-3):
        self.useBatchNorm = useBatchNorm
        self.offset = offset
        self.scale = scale
        self.var_eps = var_eps
        
    def performBatchNorm(self, matmul_plus_bias):
        if not self.useBatchNorm:
            return matmul_plus_bias
        batch_mean2, batch_var2 = tf.nn.moments(matmul_plus_bias,[0])
        return tf.nn.batch_normalization(matmul_plus_bias, batch_mean2, batch_var2, self.offset, self.scale, self.var_eps)
    
class DropoutWrapper():
    def __init__(self, useDropout = False, keep_prob = 0.9):
        self.useDropout = useDropout
        self.keep_prob = keep_prob
        
    def performDropout(self, matmul_plus_bias):
        if not self.useDropout:
            return matmul_plus_bias
        return tf.nn.dropout(matmul_plus_bias, self.keep_prob)
    
class RegularizerWrapper():
    def __init__(self, useRegularizer, scale = 0.0):
        self.useRegularizer = useRegularizer
        self.scale = scale
        
    def getRegularizer(self):
        if not self.useRegularizer:
            return None
        return tf.contrib.layers.l2_regularizer(scale=self.scale)
    
    def toStr(self):
        return str(self.scale)
    

ActivationFunctions = Enum('ActivationFunctions', 'relu tanh sigmoid')
InitializerTypes = Enum('InitializerTypes','RandomNormal RandomUniform TruncatedNormal GlorotNormal GlorotUniform')
ErrorType = Enum('ErrorType', 'RMSE MAE CrossEntropy')    

class WeightInitializerWrapper():
    def __init__(self, initializerType, 
                 rand_norm_mean = 0.0, rand_norm_stddev = 1.0, 
                 rand_uniform_min = -1.0, rand_uniform_max = 1.0,
                 trunc_norm_mean = 0.0, trunc_norm_stddev = 1.0):
        self.init = initializerType
        self.rand_norm_mean = rand_norm_mean
        self.rand_norm_stddev = rand_norm_stddev
        self.rand_uniform_min = rand_uniform_min
        self.rand_uniform_max = rand_uniform_max
        self.trunc_norm_mean = trunc_norm_mean
        self.trunc_norm_stddev = trunc_norm_stddev
        
    def getInitializer(self):
        if self.init == InitializerTypes.RandomNormal:
            return tf.random_normal_initializer(mean = self.rand_norm_mean, stddev = self.rand_norm_stddev)
        if self.init == InitializerTypes.RandomUniform:
            return tf.random_uniform_initializer(minval = self.rand_uniform_min, maxval = self.rand_uniform_max)
        if self.init == InitializerTypes.TruncatedNormal:
            return tf.truncated_normal_initializer(mean = self.trunc_norm_mean, stddev = self.trunc_norm_stddev)
        if self.init == InitializerTypes.GlorotNormal:
            return tf.glorot_normal_initializer()
        if self.init == InitializerTypes.GlorotUniform:
            return tf.glorot_uniform_initializer()
        
    def toStr(self):
        if self.init == InitializerTypes.RandomNormal:
            return 'RN' + str(self.rand_norm_stddev)
        if self.init == InitializerTypes.RandomUniform:
            return 'RU' + str(self.rand_uniform_min) + '-' + str(self.rand_uniform_max)
        if self.init == InitializerTypes.TruncatedNormal:
            return 'TN' + str(self.trunc_norm_stddev)
        if self.init == InitializerTypes.GlorotNormal:
            return 'GN'
        if self.init == InitializerTypes.GlorotUniform:
            return 'GU'
        
        
class PrintInfo():
    def __init__(self):
        pass
    
    def printEpochResult(self, epoch, validationError, trainError):
        print("======= AT EPOCH", epoch, " =========") 
        print('train error: ', trainError)
        print('validation error: ', validationError)
        
    def printSavingInfo(self, validationError):
        print('saving lower cost weights')
        print('At validation error: ', validationError)
    
    def printFinalResults(self, trainError, validationError):
        print("Training Done. Printing results...")
        print("Training Error = ", trainError) 
        print("Validation Error = ", validationError) 
    

In [ ]:
class MultiFilterSubnet():
    
    def __init__(self, numberOfFeatures, replicationCount, subnetStructure, learn_filter_contrib_weights,
                 activation, regularizer, dropout, initializer, batchNorm, isVerbose = True):
        self.numberOfFeatures = numberOfFeatures
        self.replicationCount = replicationCount
        self.subnetStructure = subnetStructure
        self.numberOfHiddenLayers = len(self.subnetStructure)-2
        self.learn_filter_contrib_weights = learn_filter_contrib_weights
        self.activation = activation
        self.isVerbose = isVerbose
        self.initializer = initializer
        self.regularizer = regularizer
        self.batchNorm = batchNorm
        self.dropout = dropout
        
        
    def runActivationFunction(self, matmul):
        if self.activation == ActivationFunctions.tanh:
            return tf.nn.tanh(matmul)
        elif self.activation == ActivationFunctions.sigmoid:
            return tf.nn.sigmoid(matmul)
        return tf.nn.relu(matmul)
    

    def makeSubnetStructure(self):   
        weights, biases = [], []
        for i in range(self.numberOfHiddenLayers + 1):
            weights.append(tf.get_variable('wt'+str(i), shape=[self.subnetStructure[i], self.subnetStructure[i+1]], 
                           initializer=self.initializer.getInitializer(), regularizer = self.regularizer.getRegularizer()))
            biases.append(tf.get_variable('bs'+str(i), shape=[self.subnetStructure[i+1]],
                                          initializer=tf.constant_initializer(0.0)))
        return weights, biases
    
    
    def build_subnet(self, subnet_input, subnetname):
        with tf.variable_scope(subnetname, reuse=True): 
            weights, biases = self.makeSubnetStructure()
            layer_output = [subnet_input]
            for i in range(self.numberOfHiddenLayers):
                matmul_plus_bias =  self.dropout.performDropout(
                                       self.runActivationFunction(
                                        self.batchNorm.performBatchNorm(
                                            tf.matmul(layer_output[i], weights[i]) + biases[i]
                                        )
                                       )
                                     )
                layer_output.append(matmul_plus_bias)
            return tf.matmul(layer_output[-1], weights[self.numberOfHiddenLayers]) + biases[self.numberOfHiddenLayers]
    
    
    def build_model(self, inputs, numberOfOutputNodes = 1):
        featureOutputs = []
        with tf.variable_scope('globwts', reuse=True):
            for i in range(self.numberOfFeatures):   
                sumSubnet = [] 
                for k in range(self.replicationCount):    
                    sumSubnet.append(self.build_subnet(inputs[i], 'subnet' + str(k)))
                    
                filterWeights = tf.get_variable('filterWeights', shape=[self.replicationCount,1], 
                                initializer=self.initializer.getInitializer(), regularizer = self.regularizer.getRegularizer())
                featureOutputs.append(tf.matmul(tf.concat(sumSubnet, 1), filterWeights)) 
        featureWeights = tf.get_variable('featureWeights', shape=[self.numberOfFeatures, numberOfOutputNodes], 
                                initializer=self.initializer.getInitializer(), regularizer = self.regularizer.getRegularizer())                                
        return tf.matmul(tf.concat(featureOutputs, 1), featureWeights)
    
    
    def getBatchData(self, train_x, train_y, batchsize, batchno):
        tx = []
        for j in range(len(train_x)):
            tx.append((np.array(train_x[j]))[batchno*batchsize: (batchno+1)*batchsize,:])
        ty = train_y[batchno*batchsize: (batchno+1)*batchsize]
        return tx, ty
    
    def getPermutedDataForIteration(self, train_x, train_y):
        tx = []
        shuffle_order = np.random.permutation(np.arange(train_y.shape[0]))
        for j in range(len(train_x)):
            tx.append((np.array(train_x[j]))[shuffle_order])
        ty = train_y[shuffle_order]
        return tx, ty


    def RunNN(self, train_x, valid_x, test_x, train_y, valid_y, learning_rate = 0.01, max_epochs = 10000, tolerance = 1e-4,
             batchsize = 32, errorType = ErrorType.RMSE, numberOfOutputNodes = 1):
        tf.reset_default_graph()
                
        with tf.variable_scope('globwts'):
            tf.get_variable('filterWeights', shape=[self.replicationCount,1], 
                                initializer=self.initializer.getInitializer(), regularizer = self.regularizer.getRegularizer())
            for i in range(self.replicationCount):
                with tf.variable_scope('subnet'+str(i)): 
                    self.makeSubnetStructure() 
                
        with tf.name_scope("IO"):
            inputarr = []
            for i in range(self.numberOfFeatures):
                inputarr.append(tf.placeholder(tf.float32, [None, self.subnetStructure[0]], name="X"+str(i)))
            outputs = tf.placeholder(tf.float32, [None, numberOfOutputNodes], name="Yhat")    

        with tf.name_scope("train"):
            yout = self.build_model(inputarr, numberOfOutputNodes) 
            cost_op = None
            if errorType == ErrorType.RMSE:
                cost_op = tf.losses.mean_squared_error(labels = outputs, predictions = yout)
            elif errorType == ErrorType.MAE:
                cost_op = tf.metrics.mean_absolute_error(labels = outputs, predictions = yout)
            elif errorType == ErrorType.CrossEntropy:
                cost_op = tf.nn.softmax_cross_entropy_with_logits(labels = outputs, logits = yout)
            else:
                raise ValueError('Inappropriate error type!')
            train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost_op)

        epoch = 0
        current_min_validation_error = 1000.0
        printInfo = PrintInfo()
        if self.isVerbose:
            print( "Beginning Training" )
        init = tf.global_variables_initializer()
        sess = tf.Session() # Create TensorFlow session
        saver = tf.train.Saver()
        save_dir = 'checkpoints/'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, 'best_validation1')
        with sess.as_default():
            sess.run(init)
            traindict = {i:t for i,t in zip(inputarr, train_x)} 
            traindict[outputs] = train_y
            validdict = {i:t for i,t in zip(inputarr, valid_x)} 
            validdict[outputs] = valid_y
            testdict = {i:t for i,t in zip(inputarr, test_x)} 
            savedEpoch = -1
            numbatch = int(train_y.shape[0]/batchsize)
            while True:                
                train_iter_x, train_iter_y = self.getPermutedDataForIteration(train_x, train_y)
                for i in range(numbatch):
                    tx, ty = self.getBatchData(train_iter_x, train_iter_y, batchsize, i)
                    tdict = {mi:mt for mi,mt in zip(inputarr, tx)} 
                    tdict[outputs] = ty
                    sess.run( train_op, feed_dict = tdict)
                    #TODO: support for classification problems
                    cost_validation = sess.run(cost_op, feed_dict = validdict)
                    if current_min_validation_error - cost_validation > tolerance:
                        if self.isVerbose == True:
                            printInfo.printSavingInfo(cost_validation)
                        saver.save(sess=sess, save_path=save_path)
                        savedEpoch = epoch
                        current_min_validation_error = cost_validation
                        
                if self.isVerbose:
                    printInfo.printEpochResult(epoch, cost_validation, sess.run(cost_op, feed_dict = traindict))
                if epoch > max_epochs :
                    break   
                epoch += 1

            saver.restore(sess=sess, save_path=save_path)
            if self.isVerbose:
                printInfo.printFinalResults(sess.run(cost_op, feed_dict= traindict), sess.run(cost_op, feed_dict= validdict))   
                
            return sess.run(yout, feed_dict=testdict), \
                      sess.run(cost_op, feed_dict= validdict), sess.run(cost_op, feed_dict= traindict), savedEpoch
        
    

In [ ]:
class SubnetWrapper():
    
    def __init__(self, num_of_features, num_of_fingerprints, replicationCount, learn_filter_contrib_weights,
                 activation, regularizer, hiddenLayer, initializer, 
                 runcnt, dropout = 1.0, learningrate = 0.001, tolerance = 1e-4):
        self.num_of_features = num_of_features
        self.num_of_fingerprints = num_of_fingerprints
        self.replicationCount = replicationCount
        self.learn_filter_contrib_weights = learn_filter_contrib_weights
        self.activation = activation
        self.regularizer = regularizer
        self.hiddenLayer = hiddenLayer
        self.initializer = initializer
        self.learningrate = learningrate
        self.runcnt = runcnt
        self.tolerance = tolerance
        if dropout > 0.99:
            self.dropout = DropoutWrapper(False)
        else:
            self.dropout = DropoutWrapper(True, dropout)
        self.categoricalVarIndices = dict()
        self.numberOfClasses = 1
    
    def extractDataColsForTF(self, data):
        x = []
        for d in range(self.num_of_features):
            x.append(data[:, d*self.num_of_fingerprints : (d+1)*self.num_of_fingerprints])
        return x
    

    def AppendListOfData(self, dataList):
        if not isinstance(dataList, list):
            if dataList.ndim == 1:
                return dataList.reshape(dataList.shape[0], 1)
            else:
                return dataList
        if len(dataList) == 0:
            print('ERROR! Empty data!!!')
            return None
        for i in range(len(dataList)):
            if dataList[i].ndim == 1:
                dataList[i] = dataList[i].reshape(dataList[i].shape[0], 1)
        appendedData = dataList[0]
        for i in range(1, len(dataList)):
            appendedData = np.vstack((appendedData, dataList[i]))
        return appendedData
    
    def convertForNN(self, X_train, X_validation, X_test, Y_train, Y_validation, Y_test, energyPivot = None):
        if Y_train.ndim == 1:
            Y_train = Y_train.reshape(Y_train.shape[0], 1)
        if Y_validation.ndim == 1:
            Y_validation = Y_validation.reshape(Y_validation.shape[0], 1)
        if Y_test.ndim == 1:
            Y_test = Y_test.reshape(Y_test.shape[0], 1)
        return self.extractDataColsForTF(X_train), \
                  self.extractDataColsForTF(X_validation),  \
                    self.extractDataColsForTF(X_test), \
                        Y_train, Y_validation, Y_test, energyPivot
                
    
    def setCategoricalVariableIndices(self, y):
        for i in range(y.shape[0]):
            if y[i,0] not in self.categoricalVarIndices:
                self.categoricalVarIndices[y[i,0]] = self.numberOfClasses
                self.numberOfClasses += 1
    
    
    def oneHotEncodingFor(self, y):
        onehotY = np.zeros((y.shape[0], self.numberOfClasses))
        for i in range(y.shape[0]):
            onehotY[i, self.categoricalVarIndices[y[i,0]]] = 1
        return onehotY
    
    
    def getOneHotEncoding(self, Y_train, Y_validation, Y_test):
        self.numberOfClasses = 0
        self.setCategoricalVariableIndices(Y_train)
        self.setCategoricalVariableIndices(Y_validation)
        self.setCategoricalVariableIndices(Y_test)
        oneHot_trainY = self.oneHotEncodingFor(Y_train)
        oneHot_validationY = self.oneHotEncodingFor(Y_validation)
        oneHot_testY = self.oneHotEncodingFor(Y_test)
        return oneHot_trainY, oneHot_validationY, oneHot_testY
    
    
    def getDataSuitableForSubnet(self, encodingTrains, energyTrains, encodingTests, energyTests, splitIndices, isCategorical):
        encodingTrain = self.AppendListOfData(encodingTrains)
        energyTrain = self.AppendListOfData(energyTrains)
        encodingTest = self.AppendListOfData(encodingTests)
        energyTest = self.AppendListOfData(energyTests)
        shuffle_order = np.random.permutation(np.arange(energyTrain.shape[0]))
        energyTrain = energyTrain[shuffle_order]
        encodingTrain = encodingTrain[shuffle_order]
        split1 = splitIndices[0]
        split2 = splitIndices[1]
        X_train, Y_train = encodingTrain[:split1, :], energyTrain[:split1]
        X_validation, Y_validation = encodingTrain[split1:split2, :], energyTrain[split1:split2]
        if isCategorical:
            Y_train, Y_validation, energyTest = self.getOneHotEncoding(Y_train, Y_validation, energyTest)        
        return self.convertForNN(X_train, X_validation, encodingTest, Y_train, Y_validation, energyTest)
        

    def prepareSubnetStructures(self, hiddenStructure):
        return [self.num_of_fingerprints] + hiddenStructure + [1]
    
    
    
    def makeRepeatedInterpolatingPrdictionsWithEnsemble(self, repeatCount, testSetSize, splitIndices, max_epochs, 
                                                        encodings, energies, batchsize = 32, errorType = ErrorType.RMSE,
                                                        isCategorical = False, useBatchNorm = False, isVerbose = True):
        errors = []
        netStruct = self.prepareSubnetStructures(self.hiddenLayer)
        print(netStruct)
        k = 0
        while k < repeatCount:
            i = 0
            test_y = None
            predsDF = pd.DataFrame(index = range(testSetSize), columns = 
                               [i for i in range(self.runcnt)] +
                              ['Mean Pred', 'Std Pred', 'Actual', 'AE'])
            encoding = self.AppendListOfData(encodings)
            energy = self.AppendListOfData(energies)
            shuffle_order = np.random.permutation(np.arange(energy.shape[0]))
            energy = energy[shuffle_order]
            encoding = encoding[shuffle_order]
            split1 = splitIndices[1]
            encoding1, energy1 = encoding[:split1, :], energy[:split1]
            encoding2, energy2 = encoding[split1:, :], energy[split1:]
            while i < self.runcnt:
                train_x, valid_x, test_x, train_y, valid_y, test_y, _ = \
                                                                            self.getDataSuitableForSubnet( \
                                                                            encoding1, energy1, encoding2, \
                                                                            energy2, splitIndices, isCategorical)
                tf.reset_default_graph()
                net = MultiFilterSubnet(self.num_of_features, self.replicationCount, netStruct, 
                                        self.learn_filter_contrib_weights, self.activation, self.regularizer, self.dropout,
                                        self.initializer, BatchNormWrapper(useBatchNorm), isVerbose = isVerbose)

                preds, verr, terr, e = net.RunNN(train_x, valid_x, test_x, train_y, valid_y, \
                                                      learning_rate = self.learningrate, max_epochs = max_epochs, \
                                                      batchsize = batchsize, tolerance = self.tolerance, 
                                                      errorType = errorType, numberOfOutputNodes = self.numberOfClasses)
                predsDF.iloc[:, i] = preds
                #TODO: support for classification problem
                testerror = np.mean(abs(preds - test_y))
                if errorType == ErrorType.RMSE:
                    testerror = np.sqrt(np.mean((preds - test_y)**2))
                    verr = np.sqrt(verr)
                    terr = np.sqrt(terr)
                print('test set error', testerror)
                print('validation error', verr)
                print('training error', terr)
                print('saved in epoch', e)
                i += 1
            predsDF.loc[:, 'Actual'] = test_y
            predictionArray = np.array(predsDF.iloc[:,:(self.runcnt)].values, dtype=np.float32)
            predsDF.loc[:, 'Mean Pred'] = np.mean(predictionArray, axis = 1)
            predsDF.loc[:, 'Std Pred'] = np.std(predictionArray, axis = 1)
            predsDF.loc[:, 'AE'] = abs(predsDF.loc[:,'Actual'] - predsDF.loc[:,'Mean Pred'])
            error = np.mean(predsDF.loc[:, 'AE'].values)
            if errorType == ErrorType.RMSE:
                error = np.sqrt(np.mean(predsDF.loc[:, 'AE'].values ** 2))
            print('Error for Ensemble:', error)
            errors.append(error)
            k += 1
            
        print('Done with mean of RMSE of %s' %(np.mean(errors)))
        return np.mean(errors), np.std(errors)
    
    


In [ ]:
import os
from datetime import datetime
import numpy as np
from pandas import pandas as pd


def RunForFilters(encoding, energy, testSetSize, splitIndices, outputDir, 
                  numOfFeatures, numOfFingerprints, hiddlayers, filterNumList, 
                  runsForFilter, runsForEnsemble, errorType, isCategorical):
    savepath = os.path.join(outputDir, 'RunResults_NewArch_NFt_' + str(numOfFeatures) + '_NFP_' + str(numOfFingerprints) +
                            '_hidd' + str(hiddlayers) + '.csv')
    collist = ['FilterCount','Mean of Error', 'SD of Error']
    if not os.path.exists(savepath):
        df = pd.DataFrame(columns = collist)
        df.to_csv(savepath)

    for k in filterNumList:
        print('Number of Filters', str(k))
        df = pd.read_csv(savepath, index_col=0)
        if len(df[(df['FilterCount']==k)]) > 0:
            print('Already computed. Continuing...')
            continue
        print('Computing...')
        pred = SubnetWrapper(num_of_features = numOfFeatures, num_of_fingerprints = numOfFingerprints, 
                                 replicationCount = k, learn_filter_contrib_weights = True,
                             activation = ActivationFunctions.tanh, regularizer = RegularizerWrapper(True, 0.001), 
                             hiddenLayer = hiddlayers, dropout = 0.9,
                          initializer = WeightInitializerWrapper(InitializerTypes.RandomNormal, rand_norm_stddev = 0.0001), 
                             learningrate = 0.001, runcnt = runsForEnsemble, tolerance = 1e-3)
        mean_error, sd_error = pred.makeRepeatedInterpolatingPrdictionsWithEnsemble(repeatCount = runsForFilter,
                                                                 testSetSize = testSetSize, 
                                                                 splitIndices = splitIndices, 
                                                                 max_epochs = 10000,
                                                                 encodings = encoding, energies = energy, 
                                                                 batchsize = 384, errorType = errorType,
                                                                 isCategorical = isCategorical,
                                                                 useBatchNorm = True, isVerbose = False)
        print('Eror for this filter:', mean_error)
        df.loc[len(df)] = dict(zip(collist, [k, mean_error, sd_error]))
        df.to_csv(savepath)
            



In [ ]:
encoding = np.loadtxt('regr_housing_attr13.csv', delimiter=',', skiprows=0, usecols=range(0,78))
energy = np.loadtxt('regr_housing_attr13.csv', delimiter=',', skiprows=0, usecols=78)
RunForFilters(encoding = encoding, energy = energy, testSetSize = 58, splitIndices = [384,448], outputDir = '', 
              numOfFeatures = 13, numOfFingerprints = 6, hiddlayers = [8,4], 
              filterNumList = [8], runsForFilter = 5, runsForEnsemble = 5, 
              errorType = ErrorType.RMSE, isCategorical = False)  

In [ ]:
encoding = np.loadtxt('ohsumed500_stat.csv', delimiter=',', skiprows=0, usecols=range(0,135))
energy = np.loadtxt('ohsumed500_stat.csv', delimiter=',', skiprows=0, usecols=135)
RunForFilters(encoding = encoding, energy = energy, testSetSize = 58, splitIndices = [400,440], outputDir = '', 
              numOfFeatures = 27, numOfFingerprints = 5, hiddlayers = [8,4], 
              filterNumList = [8], runsForFilter = 5, runsForEnsemble = 5, 
              errorType = ErrorType.CrossEntropy, isCategorical = True)  #TODO: support for classification not done yet

In [ ]:
### This cell is used to draw the weight matrix between the input layer and the first hidden layer for each subnet (filter)

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

def show_as_image(wtmatrix, numOfFilters):
    maxval = np.max(np.abs([wtmatrix.min(),wtmatrix.max()]))
    plt.figure()
    for i in range(numOfFilters):
        plt.imshow(wtmatrix[i,:,:], cmap='RdYlGn', interpolation='nearest',
                   vmin=-maxval, vmax=maxval)
        plt.colorbar()
        plt.show()
    

def makeSubnetStructure(subnetStructure):  
    weights = []
    for i in range(len(subnetStructure) - 1):
        weights.append(tf.get_variable('wt'+str(i), shape=[subnetStructure[i], subnetStructure[i+1]]))
    return weights

def getLeftWeightMatrixAndDraw(numOfFilters, subnetStructure, dirToLook, fileName):
    tf.reset_default_graph()

    with tf.variable_scope('globwts'):
        for i in range(numOfFilters):
            with tf.variable_scope('subnet'+str(i)): 
                makeSubnetStructure(subnetStructure) 

    init = tf.global_variables_initializer()
    sess = tf.Session() # Create TensorFlow session
    saver = tf.train.Saver()
    save_dir = 'checkpoints/'
    if not os.path.exists(dirToLook):
        os.makedirs(dirToLook)
    save_path = os.path.join(dirToLook, fileName)
    weightMatrix = []
    with sess.as_default():
        sess.run(init)        
        saver.restore(sess=sess, save_path=save_path)
        g = tf.get_default_graph()
        with tf.variable_scope('globwts'):
            for i in range(numOfFilters):
                with tf.variable_scope('subnet'+str(i), reuse=True):
                    weights = makeSubnetStructure(subnetStructure)
                    weightMatrix.append(weights[0].eval())

    weightMatrix = np.array(weightMatrix)
    np.save('Wtmatrix_generic', weightMatrix)
    print(weightMatrix.shape)
    show_as_image(weightMatrix, numOfFilters)
    

getLeftWeightMatrixAndDraw(numOfFilters = 8, subnetStructure = [6,8,4,1], dirToLook = 'checkpoints/', 
                           fileName = 'best_validation1')